In [2]:
import openai, json

client = openai.OpenAI()
messages = []

## 5. Function calling

In [3]:
def get_weather(city):
    return "33 degrees celcius."


def get_daily_plan(monthly_plan):
    return """
    8:00 wake up 
    8:30 breakfast
    9:00 go to work
    10:00 attendance check
    12:00 lunch
    13:00 go to work
    16:00 peer session
    17:00 snack time
    19:00 dinner
    22:15 go to bed
    """


FUNCTION_MAP = {
    "get_weather": get_weather,
    "get_daily_plan": get_daily_plan,
}

In [4]:
TOOLS = [
    {   # 첫 번째 도구 (get_daily_plan)
        "type": "function",
        "function": {
            "name": "get_daily_plan",
            "description": "A function to get the daily plan of user.",
            "parameters": {
                "type": "object",
                "properties": {
                    "monthly_plan": {
                        "type": "string",
                        "description": "A monthly plan to bring in a daily plan. monthly_plan None is possible.",
                    }
                },
                # "required": ["monthly_plan"],
            },
        },
    },
    {   # 두 번째 도구 (get_weather)
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",
                    }
                },
                "required": ["city"],
            },
        },
    }
]

In [5]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage, memory=True):
    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            },
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(
                f"""
                  Run function: {function_to_run}...\n
                  result: {result}
                """
            )

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )
        call_ai()
    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")


def send_message(memory=True) -> str:
    query = input("Send a message to the LLM : ")
    if memory:
        messages.append({"role": "user", "content": query})
    print(f"User: {query}")

    return query


def call_ai(memory=True) -> None:
    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=messages,
        tools=TOOLS,
    )

    message = response.choices[0].message

    process_ai_response(message)

In [6]:
while True:
    query = send_message()
    if query in {"quit", "q"}:
        break
    call_ai(query)

User: What is the weather in Korea?
AI: Do you mean South Korea or North Korea — and which city (for example Seoul, Busan, Incheon, Daegu, Jeju)? Also, do you want the current/today's weather or a forecast for a specific date? Tell me the city and timing and I’ll fetch it.
User: Jeju.
AI: Do you want the current weather for Jeju (Jeju City) now, or a forecast for a specific day? Which do you prefer?
User: now.
Calling function: get_weather with {"city":"Jeju"}

                  Run function: <function get_weather at 0x7f6cbe15b100>...

                  result: 33 degrees celcius.
                
AI: Current temperature in Jeju City (now): 33°C (≈91°F). 

Would you like an hourly/daily forecast, humidity/wind details, or any other info?
User: q


In [8]:
messages

[{'role': 'user', 'content': 'What is the weather in Korea?'},
 {'role': 'assistant',
  'content': "Do you mean South Korea or North Korea — and which city (for example Seoul, Busan, Incheon, Daegu, Jeju)? Also, do you want the current/today's weather or a forecast for a specific date? Tell me the city and timing and I’ll fetch it."},
 {'role': 'user', 'content': 'Jeju.'},
 {'role': 'assistant',
  'content': 'Do you want the current weather for Jeju (Jeju City) now, or a forecast for a specific day? Which do you prefer?'},
 {'role': 'user', 'content': 'now.'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_U6eSK1qURMpaM7bIzNDe3USn',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Jeju"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_U6eSK1qURMpaM7bIzNDe3USn',
  'name': 'get_weather',
  'content': '33 degrees celcius.'},
 {'role': 'assistant',
  'content': 'Current temperature in Jeju City (now): 33°C (≈91°F). \n\nWould you 